## execute this cell first
class to retrieve the sentences from the dataset

In [0]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["sentence"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("pmid").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["pmid : {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None



## read the data

In [0]:
annotated = pd.read_csv('all_annotated.csv')
data = annotated.dropna()
getter = getsentence(data)
sentences = getter.sentences

## utils functions

In [0]:
def sentense_number(abstract, token):
  blob = TextBlob(abstract)
  sous_sentenses = []
  for a in blob.sentences:
    sous_sentenses.append(str(a))

  num = sous_sentenses.index(token)+1
  return num

def sentence_position(text,token):
  indice = 0
  joker = 0
  lsx = text.split()
  chunk_size = int(len(lsx)/10)
  #print(chunk_size)
  output = [lsx[i:i+chunk_size] for i in range(0, len(lsx), chunk_size)]
  liste_des_parties = []
  for ab in output:
    part = ''
    for text in ab:
      part = part + ' ' + text
    liste_des_parties.append(part.lstrip())

  #print(liste_des_parties)
  first3 = ''
  last3 = ''
  for ele in token.split()[:3]:
    first3 = first3 + ' ' + ele
  for ele in token.split()[-3:]:
    last3 = last3 + ' ' + ele
  
  token_head = first3.lstrip()
  token_tail = last3.lstrip()
  #print(token_head)
  #print(token_tail)
  #retrunn the index of the part that contains the sentence, If after cutting a sentence straddles 2 sections, we take the max index.
  for part in liste_des_parties:
    if token in part:
      indice = liste_des_parties.index(part)+1
    elif token_tail in part:
      indice = liste_des_parties.index(part)+1
    elif token_head in part:
      joker = liste_des_parties.index(part)+1
      

  if indice == 0:
    indice = joker
  return indice

## Features

In [0]:
Ilist = ['intervention','exercise','increase','life','quality','physical','examine','health','reduce','aim','program','care','effectiveness',
'interventions','activity','fatigue','group','dietary','information', 'distress']
Mlist = ['exercise','health','outcome','physical','pain','fatigue','diet','anxiety','depression','experimental',
'body','hospital','medical','participate','blood','model','dietary','practice','vitamin', 'symptoms']
Rlist = ['group','intervention','increase','compare','effect','score','participants','change','baseline','difference','show',
'follow','time','mean','decrease','level','lower','higher','find','exercise']
Clist = ['health','suggest','benefit','decrease','supplementation','future','research','nurse','enhance','feasible',
'dietary','appear','distress','potential','diet','psychological','weight','beneficial','aerobic','approach']


def get_token_voca(token):
  i = sum(el in token.split() for el in Ilist)
  m = sum(el in token.split() for el in Mlist)
  r = sum(el in token.split() for el in Rlist)
  c = sum(el in token.split() for el in Clist)
  if i>m and i>r and i>c:
    return Ilist
  elif m>i and m>r and m>c:
    return Mlist
  elif r>i and r>m and r>c:
    return Rlist
  elif c>i and c>m and c>r:
    return Clist
  else:
    return 0


pLength, pRelLength = 0,0
nLength, nRelLength = 0,0
#les premiers mots de la phrase précédente (p0, p1)
#la longueur absolue de phrase précédente (pLength)
#la longueur relative de phrase précédente (pRelLength)
#la longueur absolue de phrase suivante (nLength)
#la longueur relative de phrase suivante (nRelLength)


def sentencefeatures(sentences,i):
  
  token = sentences[i][0].lstrip()

  features = {
      'token': token,
      'first_3_words': token.split()[:3],
      'absLength': len(token.split()),
      'lexique' : get_token_voca(token), 
      #'relLength': len(token.split())/len(abstract.split()),
      #'absNum' : sentense_number(abstract, token),
      #'relNum' : sentence_position(abstract,token)            
              
  }
  
  if i > 0:
    previous_token = sentences[i-1][0].lstrip()
    features.update({
        'firstP_2_words': previous_token.split()[:2],
        'pLength': len(previous_token.split()),
        #'pRelLength': len(previous_token.split())/len(abstract.split())
        })
  
  if i < len(sentences)-1:
    next_token = sentences[i+1][0].lstrip()
    features.update({
        'firstN_2_words': next_token.split()[:2],
        'nLength': len(next_token.split()),
        #'nRelLength': len(next_token.split())/len(abstract.split())
        })

  return features

def sent2features(sent):
    return [sentencefeatures(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [tag for sentence,tag in sent]

## Training CRF

In [0]:
pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 3.9MB/s 


In [0]:
pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
pip install python-crfsuite

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer

In [0]:
X = [sent2features(phrase) for phrase in sentences]
y = [sent2labels(phrase) for phrase in sentences] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
"""
!!! Note :  if there is an error about fitting the data , please check the output of the function get_token_voca(token),
specially this feature (  lexique' : get_token_voca(token) ) has a bad data input like None or NoType..'
"""
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',  # options: 'l2sgd', 'lbfgs', 
    c1=0.0418,           # 0.015, # not applicable for 'l2sgd'
    c2=0.00056,          # 0.0037,
    max_iterations=100, #100,
    all_possible_transitions=True,
    verbose=True
)
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 1248/1248 [00:00<00:00, 3794.39it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 51924
Seconds required: 0.076

L-BFGS optimization
c1: 0.041800
c2: 0.000560
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.10  loss=28298.67 active=51079 feature_norm=0.06
Iter 2   time=0.02  loss=27331.38 active=51100 feature_norm=0.05
Iter 3   time=0.02  loss=26915.60 active=51311 feature_norm=0.05
Iter 4   time=0.02  loss=26560.85 active=51508 feature_norm=0.06
Iter 5   time=0.02  loss=25819.18 active=51513 feature_norm=0.12
Iter 6   time=0.02  loss=25449.96 active=51669 feature_norm=0.15
Iter 7   time=0.02  loss=25283.50 active=51790 feature_norm=0.17
Iter 8   time=0.02  loss=25131.01 active=51796 feature_norm=0.18
Iter 9   time=0.02  loss=24065.68 active=51710 feature_norm=0.28
Iter 10  time=

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.0418, c2=0.00056, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=True)

In [0]:
pred = cross_val_predict(estimator=crf, X=X_test, y=y_test, cv=5)
report = flat_classification_report(y_pred=pred, y_true=y_test)
print(report)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
loading training data to CRFsuite: 100%|██████████| 250/250 [00:00<00:00, 4223.65it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 14801
Seconds required: 0.023

L-BFGS optimization
c1: 0.041800
c2: 0.000560
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.01  loss=5645.54  active=14584 feature_norm=0.06
Iter 2   time=0.01  loss=5454.66  active=14598 feature_norm=0.05
Iter 3   time=0.00  loss=5372.57  active=14504 feature_norm=0.05
Iter 4   time=0.00  loss=5301.37  active=14653 feature_norm=0.06
Iter 5   time=0.00  loss=5154.74  active=14712 feature_norm=0.11
Iter 6   time=0.00  loss=5098.34  active=14712 feature_norm=0.16
Iter 7   time=0.00  loss=5054.28  active=14763 feature_norm=0.17
Iter 8   time=0.00  loss=5020.05  active=14759 feature_norm=0.18
Iter 9   time=0.00  loss=4721.06  active=14743 feature_norm=0.32
Iter 10  time=

loading training data to CRFsuite: 100%|██████████| 250/250 [00:00<00:00, 4355.17it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 14848
Seconds required: 0.019

L-BFGS optimization
c1: 0.041800
c2: 0.000560
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.01  loss=5617.14  active=14591 feature_norm=0.06
Iter 2   time=0.00  loss=5399.68  active=14629 feature_norm=0.05
Iter 3   time=0.00  loss=5321.94  active=14701 feature_norm=0.04
Iter 4   time=0.00  loss=5260.42  active=14735 feature_norm=0.05
Iter 5   time=0.00  loss=5080.62  active=14740 feature_norm=0.12
Iter 6   time=0.00  loss=5035.18  active=14803 feature_norm=0.16
Iter 7   time=0.00  loss=5006.96  active=14817 feature_norm=0.17
Iter 8   time=0.00  loss=4983.68  active=14810 feature_norm=0.18
Iter 9   time=0.00  loss=4884.25  active=14789 feature_norm=0.22
Iter 10  time=

loading training data to CRFsuite: 100%|██████████| 250/250 [00:00<00:00, 4370.14it/s]


...

loading training data to CRFsuite: 100%|██████████| 251/251 [00:00<00:00, 4285.74it/s]

Iter 87  time=0.01  loss=119.93   active=3271  feature_norm=69.04
Iter 88  time=0.01  loss=119.56   active=3268  feature_norm=69.09
Iter 89  time=0.00  loss=119.40   active=3255  feature_norm=69.07
Iter 90  time=0.00  loss=119.16   active=3234  feature_norm=69.09
Iter 91  time=0.00  loss=118.95   active=3228  feature_norm=69.11
Iter 92  time=0.00  loss=118.65   active=3204  feature_norm=69.12
Iter 93  time=0.00  loss=118.42   active=3219  feature_norm=69.22
Iter 94  time=0.00  loss=118.11   active=3162  feature_norm=69.26
Iter 95  time=0.00  loss=117.82   active=3083  feature_norm=69.40
Iter 96  time=0.00  loss=117.60   active=3045  feature_norm=69.43
Iter 97  time=0.00  loss=117.38   active=3008  feature_norm=69.46
Iter 98  time=0.00  loss=117.10   active=2975  feature_norm=69.58
Iter 99  time=0.00  loss=116.92   active=2951  feature_norm=69.68
Iter 100 time=0.00  loss=116.75   active=2941  feature_norm=69.75
L-BFGS terminated with the maximum number of iterations
Total seconds requir


loading training data to CRFsuite: 100%|██████████| 251/251 [00:00<00:00, 4292.15it/s]

Iter 51  time=0.01  loss=140.94   active=6024  feature_norm=61.69
Iter 52  time=0.00  loss=138.85   active=5738  feature_norm=61.77
Iter 53  time=0.00  loss=137.49   active=5653  feature_norm=61.91
Iter 54  time=0.00  loss=135.85   active=5447  feature_norm=62.43
Iter 55  time=0.00  loss=134.24   active=5264  feature_norm=63.19
Iter 56  time=0.00  loss=132.74   active=5121  feature_norm=64.06
Iter 57  time=0.00  loss=131.43   active=4981  feature_norm=64.83
Iter 58  time=0.00  loss=130.04   active=4745  feature_norm=65.64
Iter 59  time=0.00  loss=128.95   active=4575  feature_norm=66.41
Iter 60  time=0.00  loss=128.04   active=4438  feature_norm=66.72
Iter 61  time=0.00  loss=127.00   active=4324  feature_norm=66.93
Iter 62  time=0.00  loss=126.14   active=4170  feature_norm=67.36
Iter 63  time=0.00  loss=125.37   active=4052  feature_norm=67.44
Iter 64  time=0.00  loss=124.78   active=4016  feature_norm=67.56
Iter 65  time=0.00  loss=123.96   active=3902  feature_norm=67.65
Iter 66  t

Iter 35  time=0.01  loss=419.31   active=13571 feature_norm=37.36
Iter 36  time=0.01  loss=340.62   active=13451 feature_norm=40.91
Iter 37  time=0.00  loss=297.92   active=13223 feature_norm=44.91
Iter 38  time=0.00  loss=253.76   active=12440 feature_norm=48.60
Iter 39  time=0.00  loss=219.37   active=11729 feature_norm=52.18
Iter 40  time=0.00  loss=197.96   active=11025 feature_norm=54.94
Iter 41  time=0.01  loss=192.83   active=10846 feature_norm=55.34
Iter 42  time=0.00  loss=179.45   active=10815 feature_norm=56.74
Iter 43  time=0.00  loss=171.67   active=9980  feature_norm=57.36
Iter 44  time=0.01  loss=167.24   active=9899  feature_norm=57.99
Iter 45  time=0.00  loss=159.59   active=9239  feature_norm=58.40
Iter 46  time=0.00  loss=154.15   active=9047  feature_norm=58.52
Iter 47  time=0.00  loss=147.44   active=7661  feature_norm=59.21
Iter 48  time=0.00  loss=144.57   active=7429  feature_norm=59.78
Iter 49  time=0.00  loss=142.03   active=7377  feature_norm=60.07
Iter 50  t

## Visualisation
display the most important functionalities for your model (here we have chosen 30), also the transitions between the sections.

In [0]:
import eli5
eli5.show_weights(crf, top=30)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a p

### - we show here for each functionality its importance in relation to all the sections.
### - to view all the features, please put the first word of each feature in the variable "**feature_re**" and **execute** as long as the number of features in the model.

In [0]:
eli5.show_weights(crf, top=10, feature_re='^l',
                  horizontal_layout=False, show=['targets'])

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


## Evaluation